In [102]:
import cudaq

import numpy as np
import scipy as sp
import scipy.stats as stats

import sympy as smp

In [103]:
def linear_indep(*mats: np.ndarray):
    vecs = [m.flatten() for m in mats]

    M = np.column_stack(vecs)
    rank = np.linalg.matrix_rank(M)
    
    return rank == len(mats)

In [ ]:
X = np.array([
    [0, 1],
    [0, 0]
])

Y = np.array([
    [0, 0],
    [1, 0]
])

op_set = [X, Y]

# commutate
def commutator(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    return A @ B - B @ A

def single_pass(op_set: list[np.ndarray]) -> tuple[list[np.ndarray], int]:
    new_ops = []
    n = len(op_set)

    # single pass to calculate commutators
    for i in range(n):
        for j in range(i+1, n):
            H = commutator(op_set[i], op_set[j])

            # continue if 0 matrix
            if np.allclose(H, 0): 
                continue

            print(f"Calculating [op[{i}], op[{j}]]")

            curr = [m.flatten() for m in (op_set + new_ops)]
            new = curr + [H.flatten()]
            rank_curr, rank_new = np.linalg.matrix_rank(curr), np.linalg.matrix_rank(new)

            if rank_new > rank_curr:
                new_ops.append(H)

    return new_ops

def compute_dla(*generators: np.ndarray, lim=20):
    basis = list(generators)

    i = 0
    while True:
        print(f"Iteration: {i}, basis size of {len(basis)}")

        new_ops = single_pass(basis)

        if len(new_ops) == 0:
            break

        basis.extend(new_ops)
        i+=1
        
        # with np.printoptions(precision=2, suppress=True, linewidth=1000):
        for new_op in new_ops:
            print(f"Added\n{new_op}")

        if i > lim:
            print(f"iterations exceeded {lim}, DLA loop terminated")
            break

    rank = np.linalg.matrix_rank([m.flatten() for m in basis])
    return basis, rank

dla = compute_dla(X, Y)
dla


Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[ 1  0]
 [ 0 -1]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]


([array([[0, 1],
         [0, 0]]),
  array([[0, 0],
         [1, 0]]),
  array([[ 1,  0],
         [ 0, -1]])],
 np.int64(3))

In [105]:
X_3 = np.array([
    [0, 0, 1],
    [0, 0, 0],
    [0, 0, 0]
])

Y_3 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [1, 0, 0]
])

dla = compute_dla(X_3, Y_3)

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[ 1  0  0]
 [ 0  0  0]
 [ 0  0 -1]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]


In [106]:
X = stats.unitary_group.rvs(dim=3)
Y = stats.unitary_group.rvs(dim=3)

dla = compute_dla(X, Y)
dla

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[ 0.21849754+0.34595743j  0.86882361-0.76546547j  0.35546733-0.79629769j]
 [ 0.41295676+0.2797773j  -0.22901657-0.24744534j -1.20794818-0.64297989j]
 [ 0.98490644+1.12488661j -0.24391389+0.92075002j  0.01051902-0.09851209j]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]
Added
[[-0.1093473 +0.06739331j -1.16268565-0.50404864j -2.87095518-0.31178681j]
 [-1.30301572+1.68141966j -0.5616266 +0.503074j    0.5723907 +0.99628852j]
 [-0.88911631+1.26893268j -1.93177505+0.82949006j  0.6709739 -0.57046731j]]
Added
[[-0.76706808+1.49630406j -1.45996673-0.24336367j -1.52042008+0.37288734j]
 [ 2.57691165+1.31803434j -0.07748835-0.18192783j -0.62515792-0.83498557j]
 [-0.18576615-0.04641888j -0.48995068-2.13545559j  0.84455643-1.31437623j]]
Iteration: 2, basis size of 5
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[0], op[3]]
Calculating [op[0], op[4]

([array([[-0.08779438-0.63389508j,  0.63724213-0.39059428j,
           0.11626223+0.13531753j],
         [ 0.37216036-0.06755177j, -0.3643673 -0.1957736j ,
           0.42448239+0.71109584j],
         [-0.50288189+0.4410356j ,  0.41377555+0.31473406j,
           0.36576621+0.38541503j]]),
  array([[-0.23084792-0.05725576j,  0.75737466-0.3022165j ,
          -0.20979642+0.48421616j],
         [ 0.44440151-0.10052558j,  0.50174978+0.12222357j,
           0.69472348-0.20753251j],
         [ 0.82428208-0.23746994j, -0.12986312-0.22692056j,
          -0.34523663+0.27680567j]]),
  array([[ 0.21849754+0.34595743j,  0.86882361-0.76546547j,
           0.35546733-0.79629769j],
         [ 0.41295676+0.2797773j , -0.22901657-0.24744534j,
          -1.20794818-0.64297989j],
         [ 0.98490644+1.12488661j, -0.24391389+0.92075002j,
           0.01051902-0.09851209j]]),
  array([[-0.1093473 +0.06739331j, -1.16268565-0.50404864j,
          -2.87095518-0.31178681j],
         [-1.30301572+1.68141966j,

In [110]:
I = np.array([
    [1, 0],
    [0, 1]
])

X = np.array([
    [0, 1],
    [1, 0]
])
Y = np.array([
    [0, -1j],
    [1j, 0]
])
Z = np.array([
    [1, 0],
    [0, -1]
])

dla, dim = compute_dla(np.kron(X, I), np.kron(Y, I))
dla, dim

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[0.+2.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+2.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-2.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-2.j]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]


([array([[0, 0, 1, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0],
         [0, 1, 0, 0]]),
  array([[0.+0.j, 0.+0.j, 0.-1.j, 0.-0.j],
         [0.+0.j, 0.+0.j, 0.-0.j, 0.-1.j],
         [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j]]),
  array([[0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j]])],
 np.int64(3))